# _IRA Tweets: Data Wrangling_

In [5]:
import pandas as pd
pd.set_option("display.max_columns", None)

dtype = {
    "tweetid": str
}

cols_interest = [
    "tweetid",
    "user_screen_name",
    "tweet_time",
    "tweet_language",
    "tweet_client_name",
    "retweet_count",
    "like_count",
    "tweet_text"
]

# creater object for iteration with chunksize of 50,000 observations
chunks = pd.read_csv(
    "tr-data/ira_tweets_csv_hashed.csv",
    usecols=cols_interest,
    chunksize=50000,
    dtype=dtype,
    low_memory=False)

In [6]:
from tqdm import tqdm

# empty list to store chunks
chunk_list = []

# iterate through chunks, appending to list
for chunk in tqdm(chunks):
    chunk_list.append(chunk)
    
# concat list into dataframe
df = pd.concat(chunk_list)

176it [02:00,  1.46it/s]


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8768633 entries, 0 to 8768632
Data columns (total 8 columns):
tweetid              object
user_screen_name     object
tweet_language       object
tweet_text           object
tweet_time           object
tweet_client_name    object
like_count           float64
retweet_count        float64
dtypes: float64(2), object(6)
memory usage: 535.2+ MB


## _Check languages_

In [8]:
# rearrange columns
df = df[cols_interest]

In [11]:
# see value counts in tweet_language column
df["tweet_language"].value_counts()

ru     4853185
en     2997181
und     229392
de       98460
uk       82237
bg       54690
ar       37241
it       20136
fr       12241
es       11922
in        9523
sr        8045
tl        6618
ht        6170
et        5847
tr        3832
sk        3343
nl        3151
da        2987
ro        2980
pt        2698
ja        2653
cy        2591
sl        2307
pl        1804
sv        1759
fi        1611
no        1500
lt        1340
cs        1037
bs         615
lv         505
hu         499
id         466
hi         307
hr         297
ko         288
vi         283
is         268
eu         223
zh          56
iw          56
th          40
fa          37
el          35
hy          23
ur          17
ne           7
si           5
ta           4
bn           3
km           3
ml           2
mr           2
he           2
ug           1
iu           1
ps           1
Name: tweet_language, dtype: int64

In [12]:
# retrieve only observations in English (i.e. en)
english = df[df["tweet_language"] == "en"]

In [16]:
# reset index of English-only dataframe
english = english.reset_index(drop=True)

## _Investigate `retweet_count`_

In [17]:
# see how many missing values there are in retweet_count column
english["retweet_count"].isna().sum()

2310

In [18]:
english[english["retweet_count"].isna()][:3]

,tweetid,user_screen_name,tweet_time,tweet_language,tweet_client_name,retweet_count,like_count,tweet_text
151,886061215251693569,a95a911dd6ae864c48ed062cdbe75e5c28dbe0cf57c6db...,2017-07-15 03:13,en,Twitter for Android,NaN,NaN,RT @W_C_Patriot: A strong man can handle a str...
1063,783688055697707009,b61d47c22efd69b47675c296d5c54e2675da8977504225...,2016-10-05 15:19,en,Twitter Web Client,NaN,NaN,"RT @TheZenGardner: Raw oysters? Yes, please. ..."
5550,884606889232744448,a95a911dd6ae864c48ed062cdbe75e5c28dbe0cf57c6db...,2017-07-11 02:54,en,Twitter for Android,NaN,NaN,RT @Cernovich: Instead of covering ANTIFA viol...


In [19]:
english[english["retweet_count"].isna()][-3:]

,tweetid,user_screen_name,tweet_time,tweet_language,tweet_client_name,retweet_count,like_count,tweet_text
2991950,829369918567092224,aacff3990c798063b89807b9e0f7bebf225930adaca5a7...,2017-02-08 16:42,en,Twitter Web Client,NaN,NaN,RT @facebookie: #ValentinesDayIn3Words Hallmar...
2995329,931584081652879361,a95a911dd6ae864c48ed062cdbe75e5c28dbe0cf57c6db...,2017-11-17 18:05,en,Twitter for Android,NaN,NaN,RT @Cernovich: Democrats are the real sexists.
2996354,673891173627367425,GiselleEvns,2015-12-07 15:45,en,Twitter Web Client,NaN,NaN,RT @demonwheels666: #RejectedChristmasDecorati...


In [24]:
english["retweet_count"].value_counts(dropna=False)[:15]

0.0     2572170
1.0      176527
2.0       65334
3.0       31715
4.0       17891
5.0       12273
6.0        9036
7.0        6915
8.0        4812
9.0        4047
10.0       3426
11.0       3199
12.0       2893
13.0       2620
14.0       2402
NaN        2310
15.0       2174
Name: retweet_count, dtype: int64

In [25]:
# replace NaNs in retweet_count column with 0
english["retweet_count"] = english["retweet_count"].fillna(0)

In [26]:
# confirm that it worked
english["retweet_count"].value_counts(dropna=False)[:15]

0.0     2574480
1.0      176527
2.0       65334
3.0       31715
4.0       17891
5.0       12273
6.0        9036
7.0        6915
8.0        4812
9.0        4047
10.0       3426
11.0       3199
12.0       2893
13.0       2620
14.0       2402
15.0       2174
Name: retweet_count, dtype: int64

In [27]:
# convert retweet_count to int type
english["retweet_count"] = english["retweet_count"].astype(int)

## _Investigate `like_count`_

In [29]:
# see how many missing values there are in retweet_count column
english["like_count"].isna().sum()

2310

In [30]:
# check out first three observations with NaNs in like_count
english[english["like_count"].isna()][:3]

,tweetid,user_screen_name,tweet_time,tweet_language,tweet_client_name,retweet_count,like_count,tweet_text
151,886061215251693569,a95a911dd6ae864c48ed062cdbe75e5c28dbe0cf57c6db...,2017-07-15 03:13,en,Twitter for Android,0,NaN,RT @W_C_Patriot: A strong man can handle a str...
1063,783688055697707009,b61d47c22efd69b47675c296d5c54e2675da8977504225...,2016-10-05 15:19,en,Twitter Web Client,0,NaN,"RT @TheZenGardner: Raw oysters? Yes, please. ..."
5550,884606889232744448,a95a911dd6ae864c48ed062cdbe75e5c28dbe0cf57c6db...,2017-07-11 02:54,en,Twitter for Android,0,NaN,RT @Cernovich: Instead of covering ANTIFA viol...


In [31]:
# check out last three observations with NaNs in like_count
english[english["like_count"].isna()][-3:]

,tweetid,user_screen_name,tweet_time,tweet_language,tweet_client_name,retweet_count,like_count,tweet_text
2991950,829369918567092224,aacff3990c798063b89807b9e0f7bebf225930adaca5a7...,2017-02-08 16:42,en,Twitter Web Client,0,NaN,RT @facebookie: #ValentinesDayIn3Words Hallmar...
2995329,931584081652879361,a95a911dd6ae864c48ed062cdbe75e5c28dbe0cf57c6db...,2017-11-17 18:05,en,Twitter for Android,0,NaN,RT @Cernovich: Democrats are the real sexists.
2996354,673891173627367425,GiselleEvns,2015-12-07 15:45,en,Twitter Web Client,0,NaN,RT @demonwheels666: #RejectedChristmasDecorati...


In [34]:
# check out value counts for like_count column
english["like_count"].value_counts(dropna=False)[:15]

0.0     2569876
1.0      200943
2.0       61404
3.0       25726
4.0       14506
5.0        9715
6.0        7142
7.0        5441
8.0        4707
9.0        3837
10.0       3360
11.0       2922
12.0       2662
13.0       2330
NaN        2310
14.0       2102
15.0       1956
Name: like_count, dtype: int64

In [35]:
# fill NaNs in like_count column with 0
english["like_count"] = english["like_count"].fillna(0)

# convert column to int type
english["like_count"] = english["like_count"].astype(int)

In [36]:
# check out value counts for like_count column
english["like_count"].value_counts(dropna=False)[:15]

0     2572186
1      200943
2       61404
3       25726
4       14506
5        9715
6        7142
7        5441
8        4707
9        3837
10       3360
11       2922
12       2662
13       2330
14       2102
Name: like_count, dtype: int64

In [37]:
# check all columns for missing values
english.isna().sum()

tweetid                0
user_screen_name       0
tweet_time             0
tweet_language         0
tweet_client_name    224
retweet_count          0
like_count             0
tweet_text             0
dtype: int64

## _Investigate `tweet_client_name` column & address missing values_

In [39]:
english[english["tweet_client_name"].isna()][:3]

,tweetid,user_screen_name,tweet_time,tweet_language,tweet_client_name,retweet_count,like_count,tweet_text
38331,378110495888134144,c6fdb5b31b765db7e3378b1e23b86ea2b0eb3c03e8db56...,2013-09-12 10:59,en,NaN,0,0,RT @RobertoBella01: It's a beautiful day !!! :...
49777,351307531605381120,c6fdb5b31b765db7e3378b1e23b86ea2b0eb3c03e8db56...,2013-06-30 11:53,en,NaN,0,0,RT @MomsenG: @c6fdb5b31b765db7e3378b1e23b86ea2...
76365,366871831950536705,c6fdb5b31b765db7e3378b1e23b86ea2b0eb3c03e8db56...,2013-08-12 10:40,en,NaN,0,0,RT @Bellarosa_Fan: We love you! http://t.co/7m...


In [40]:
english[english["tweet_client_name"].isna()][-3:]

,tweetid,user_screen_name,tweet_time,tweet_language,tweet_client_name,retweet_count,like_count,tweet_text
2978388,358977873996034051,c6fdb5b31b765db7e3378b1e23b86ea2b0eb3c03e8db56...,2013-07-21 15:52,en,NaN,0,0,RT @Sandruschkaa: Over and over again http://t...
2988038,368774916373356544,c6fdb5b31b765db7e3378b1e23b86ea2b0eb3c03e8db56...,2013-08-17 16:42,en,NaN,0,0,RT @RobertoBella01: Love u all of you ! #foll...
2990029,347027076760608768,8e0f9e0d48b59943194b3bed56e1b910c1ef89adfe59e3...,2013-06-18 16:24,en,NaN,0,0,RT @idyduven: is moving to their own private i...


In [43]:
# fill in missing values with unknown string
english["tweet_client_name"] = english["tweet_client_name"].fillna("unknown")

In [50]:
# check value counts for client column
len(english[english["tweet_client_name"] == "unknown"])

224

In [51]:
# check all columns for missing values
english.isna().sum()

tweetid              0
user_screen_name     0
tweet_time           0
tweet_language       0
tweet_client_name    0
retweet_count        0
like_count           0
tweet_text           0
dtype: int64

## _Check for Retweets_

In [53]:
# see how many observations start with RT @ (indicating that it is a retweet)
english["tweet_text"].str.startswith("RT @").sum()

1090542

In [54]:
# see how many observations contain RT @ (indicating either retweet or quote tweet)
english["tweet_text"].str.contains("RT @").sum()

1091134

In [55]:
english["tweet_text"].str.contains("RT @").sum() - english["tweet_text"].str.startswith("RT @").sum()

592

In [56]:
# create subset that is only English and doesn't have retweets
eng_noretweet = english[~english["tweet_text"].str.startswith("RT @")]

In [57]:
# get info on subset we just created
eng_noretweet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1906639 entries, 0 to 2997180
Data columns (total 8 columns):
tweetid              object
user_screen_name     object
tweet_time           object
tweet_language       object
tweet_client_name    object
retweet_count        int64
like_count           int64
tweet_text           object
dtypes: int64(2), object(6)
memory usage: 130.9+ MB


In [58]:
# reset the index
eng_noretweet = eng_noretweet.reset_index(drop=True)

In [59]:
eng_noretweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1906639 entries, 0 to 1906638
Data columns (total 8 columns):
tweetid              object
user_screen_name     object
tweet_time           object
tweet_language       object
tweet_client_name    object
retweet_count        int64
like_count           int64
tweet_text           object
dtypes: int64(2), object(6)
memory usage: 116.4+ MB


## _See how many unique values are in each column_

In [63]:
for col in list(eng_noretweet.columns):
    print(f"{col} unique values -->", len(eng_noretweet[col].unique()))

tweetid unique values --> 1906639
user_screen_name unique values --> 2536
tweet_time unique values --> 810888
tweet_language unique values --> 1
tweet_client_name unique values --> 117
retweet_count unique values --> 3103
like_count unique values --> 3554
tweet_text unique values --> 1448005


## _Clean up text_

In [64]:
import re

def text_clean(text):
    """
    Function for cleaning Tweet text.
    """
    # eliminate newline characters
    text = text.strip("\n\n")
    text = text.strip("\n")
    # delete links and any alphanumeric character and the underscore, strips whitespace
    text = re.sub(r"http\S+|[^\w\s]", "", text).strip(" ")
    # replaces any non-alphanumeric character with blank space
    text = re.sub("\W", " ", text)
    # removes any extra whitespace
    text = re.sub("\s+", " ", text)
    return text

In [75]:
# pick out 5 random observation from data
five = eng_noretweet.sample(n=5, random_state=1)

In [76]:
# inspect sample observation
five

,tweetid,user_screen_name,tweet_time,tweet_language,tweet_client_name,retweet_count,like_count,tweet_text
1699076,591646222735245312,c414f359478dd67d4b86a8f80d1ce208325a42a2489304...,2015-04-24 16:53,en,Twitter Web Client,0,0,I`M ON THE HIGHWAY TO HELL! http://t.co/P0XnQI...
1048904,772757830411563012,9e4bae21f00d13160fea0fc876fbff777c2400832ae049...,2016-09-05 11:26,en,Twitter Web Client,0,0,#GameShowHostPickUpLines For a consolation pri...
572336,505994910034841600,c59c83796e1eff713d2dd97dd1b5967b22d513f79f7edb...,2014-08-31 08:26,en,vavilonX,0,0,@06dd583d55075eb926897ef48a2415afc1776718c86b6...
734039,758054735509270530,a95a911dd6ae864c48ed062cdbe75e5c28dbe0cf57c6db...,2016-07-26 21:41,en,Twitter for Android,0,0,Especially @KatyPerry who is terrible. https:/...
761810,500541983421919232,8b41afdc536a9ad49d30b9c2a845c479239259e7266653...,2014-08-16 07:18,en,vavilonX,0,0,@17554dad557498db58086f7df493af38e4cb4e8a124b6...


In [77]:
# print out text
five.loc[1699076, "tweet_text"]

'I`M ON THE HIGHWAY TO HELL! http://t.co/P0XnQIIopv'

In [78]:
text_clean(five["tweet_text"][1699076])

'IM ON THE HIGHWAY TO HELL'

In [80]:
# generate new clean text column
five["clean_text"] = five["tweet_text"].apply(lambda x: text_clean(x))

In [84]:
five.index

Int64Index([1699076, 1048904, 572336, 734039, 761810], dtype='int64')

In [86]:
# compare 
five.loc[list(five.index), ["tweet_text", "clean_text"]]

,tweet_text,clean_text
1699076,I`M ON THE HIGHWAY TO HELL! http://t.co/P0XnQI...,IM ON THE HIGHWAY TO HELL
1048904,#GameShowHostPickUpLines For a consolation pri...,GameShowHostPickUpLines For a consolation priz...
572336,@06dd583d55075eb926897ef48a2415afc1776718c86b6...,06dd583d55075eb926897ef48a2415afc1776718c86b6a...
734039,Especially @KatyPerry who is terrible. https:/...,Especially KatyPerry who is terrible
761810,@17554dad557498db58086f7df493af38e4cb4e8a124b6...,17554dad557498db58086f7df493af38e4cb4e8a124b60...


In [88]:
for i, row in five.iterrows():
    print(row["tweet_text"], "\n")
    print(row["clean_text"], "\n", "-"*30)

I`M ON THE HIGHWAY TO HELL! http://t.co/P0XnQIIopv 

IM ON THE HIGHWAY TO HELL 
 ------------------------------
#GameShowHostPickUpLines For a consolation prize you've won a weekend at casa de la me. 

GameShowHostPickUpLines For a consolation prize youve won a weekend at casa de la me 
 ------------------------------
@06dd583d55075eb926897ef48a2415afc1776718c86b6ac5482e086a0e6d75d2 Your brain needs some assistance 

06dd583d55075eb926897ef48a2415afc1776718c86b6ac5482e086a0e6d75d2 Your brain needs some assistance 
 ------------------------------
Especially @KatyPerry who is terrible. https://t.co/GfMoSuY7ce 

Especially KatyPerry who is terrible 
 ------------------------------
@17554dad557498db58086f7df493af38e4cb4e8a124b606f3c1f1cba59b76894 If only it weren't true. 

17554dad557498db58086f7df493af38e4cb4e8a124b606f3c1f1cba59b76894 If only it werent true 
 ------------------------------


## _Clean `tweet_text` column for entire data set_

In [89]:
%%time

# generate new clean text column
eng_noretweet["clean_text"] = eng_noretweet["tweet_text"].apply(lambda x: text_clean(x))

CPU times: user 25.3 s, sys: 1.19 s, total: 26.5 s
Wall time: 26.6 s


In [90]:
# check out df
eng_noretweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1906639 entries, 0 to 1906638
Data columns (total 9 columns):
tweetid              object
user_screen_name     object
tweet_time           object
tweet_language       object
tweet_client_name    object
retweet_count        int64
like_count           int64
tweet_text           object
clean_text           object
dtypes: int64(2), object(7)
memory usage: 130.9+ MB


## _Rename columns_

In [91]:
# list of new column names (which are the same ones for Verified data sets)
new_col_names = [
    "id_str",
    "screen_name",
    "created_at",
    "lang",
    "source",
    "retweet_count",
    "favorite_count",
    "full_text",
    "clean_text"
]

# assign new column names
eng_noretweet.columns = new_col_names

In [92]:
eng_noretweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1906639 entries, 0 to 1906638
Data columns (total 9 columns):
id_str            object
screen_name       object
created_at        object
lang              object
source            object
retweet_count     int64
favorite_count    int64
full_text         object
clean_text        object
dtypes: int64(2), object(7)
memory usage: 130.9+ MB


## _Add Label column_

In [93]:
# create new label column (indicating it is from a fake account)
eng_noretweet["label"] = "fake"

In [94]:
eng_noretweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1906639 entries, 0 to 1906638
Data columns (total 10 columns):
id_str            object
screen_name       object
created_at        object
lang              object
source            object
retweet_count     int64
favorite_count    int64
full_text         object
clean_text        object
label             object
dtypes: int64(2), object(8)
memory usage: 145.5+ MB


## _Shuffle/Split Data into Train & Test sets_

In [95]:
from sklearn.model_selection import train_test_split

In [96]:
train, test = train_test_split(eng_noretweet, test_size=0.2, random_state=1, shuffle=True)

In [97]:
len(train), len(test)

(1525311, 381328)

In [98]:
train = train.reset_index(drop=True)
train[:5]

,id_str,screen_name,created_at,lang,source,retweet_count,favorite_count,full_text,clean_text,label
0,536854020430639105,b0e81e4e718dfebdcd4c8601f251963b8859b155d59ad6...,2014-11-24 12:09,en,Twitter Web Client,0,0,turn down the love songs that you hear 'cause ...,turn down the love songs that you hear cause y...,fake
1,596351085058478080,KansasDailyNews,2015-05-07 16:29,en,twitterfeed,0,0,Pop-up storms possible this afternoon #local,Popup storms possible this afternoon local,fake
2,687859001699139585,b4bc6c5b9477e02b21ade00847c3d6f2dc04acb0987cbb...,2016-01-15 04:49,en,Twitter Web Client,0,0,Too bored? Watch GOP debate tonight! You'll la...,Too bored Watch GOP debate tonight Youll laugh...,fake
3,898360700086747136,306fa811b7752ad7737c5acd42395e999e5a8fb72a3a83...,2017-08-18 01:47,en,IFTTT,0,0,Man-made global warming is still a myth even t...,Manmade global warming is still a myth even th...,fake
4,530379847747383296,138c356cde5939060f3491ce10e756cadb57274aba02e9...,2014-11-06 15:23,en,Twitter Web Client,0,0,"here we are now, entertain us",here we are now entertain us,fake


In [100]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1525311 entries, 0 to 1525310
Data columns (total 10 columns):
id_str            1525311 non-null object
screen_name       1525311 non-null object
created_at        1525311 non-null object
lang              1525311 non-null object
source            1525311 non-null object
retweet_count     1525311 non-null int64
favorite_count    1525311 non-null int64
full_text         1525311 non-null object
clean_text        1525311 non-null object
label             1525311 non-null object
dtypes: int64(2), object(8)
memory usage: 116.4+ MB


In [99]:
test = test.reset_index(drop=True)
test[:5]

,id_str,screen_name,created_at,lang,source,retweet_count,favorite_count,full_text,clean_text,label
0,591646222735245312,c414f359478dd67d4b86a8f80d1ce208325a42a2489304...,2015-04-24 16:53,en,Twitter Web Client,0,0,I`M ON THE HIGHWAY TO HELL! http://t.co/P0XnQI...,IM ON THE HIGHWAY TO HELL,fake
1,772757830411563012,9e4bae21f00d13160fea0fc876fbff777c2400832ae049...,2016-09-05 11:26,en,Twitter Web Client,0,0,#GameShowHostPickUpLines For a consolation pri...,GameShowHostPickUpLines For a consolation priz...,fake
2,505994910034841600,c59c83796e1eff713d2dd97dd1b5967b22d513f79f7edb...,2014-08-31 08:26,en,vavilonX,0,0,@06dd583d55075eb926897ef48a2415afc1776718c86b6...,06dd583d55075eb926897ef48a2415afc1776718c86b6a...,fake
3,758054735509270530,a95a911dd6ae864c48ed062cdbe75e5c28dbe0cf57c6db...,2016-07-26 21:41,en,Twitter for Android,0,0,Especially @KatyPerry who is terrible. https:/...,Especially KatyPerry who is terrible,fake
4,500541983421919232,8b41afdc536a9ad49d30b9c2a845c479239259e7266653...,2014-08-16 07:18,en,vavilonX,0,0,@17554dad557498db58086f7df493af38e4cb4e8a124b6...,17554dad557498db58086f7df493af38e4cb4e8a124b60...,fake


In [101]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381328 entries, 0 to 381327
Data columns (total 10 columns):
id_str            381328 non-null object
screen_name       381328 non-null object
created_at        381328 non-null object
lang              381328 non-null object
source            381328 non-null object
retweet_count     381328 non-null int64
favorite_count    381328 non-null int64
full_text         381328 non-null object
clean_text        381328 non-null object
label             381328 non-null object
dtypes: int64(2), object(8)
memory usage: 29.1+ MB


## _Create JSONs of train and test data_

In [102]:
train.to_json("ira_train.json", orient="split")

In [103]:
test.to_json("ira_test.json", orient="split")